# Mathematical Framework
Model Assumptions:
1. Gene cassettes are in a plasmid and there are no long-range structural interactions.
2. There are no side reactions between species.
3. Geneandpromoterconcentrationisassumedtocontributeanegligibleimpactontheexpression
of the gene[20] and are therefore modelled as constant.
1. Concentrations of RNAP and ribosome are relatively low[9]
2. The behaviour of the system can be approximated by the quasi-steady state (QSS) assumption.
3. The timescale of the system is short therefore dilution effects on mRNA can be absorbed into
the mRNA decay rate[9].
1. There is no degradation of protein (although it can be easily added to the dilution term).
2. Growth rate is constant to model the average rate of dilution.
3. Each promoter can only be activated or repressed by one TF of varying cooperativities.
4.  Combinatorial promoters exhibit competitive binding of TFs but only can only bind to one activator and one repressor (although it can be extended).
5.  There is assumed to be no ”leakiness” to minimise the number of parameters.
## Ta